In [1]:
ALL_CARD_TYPES = ["Wizard", "Ghost","Knight","Firespell","plant","skeleton","HealSpell","ExorcismSpell","Executioner"]
CARD_TO_INDEX = {name: i for i, name in enumerate(ALL_CARD_TYPES)}
def encode_cards(card_list):
    vec = [0] * len(ALL_CARD_TYPES)
    for c in card_list:
        if isinstance(c, str):  # already a name
            idx = CARD_TO_INDEX.get(c, None)
        else:  # card object
            idx = CARD_TO_INDEX.get(c.__class__.__name__, None)
        if idx is not None:
            vec[idx] = 1
    return vec
import numpy as np

def snapshot_to_vector(snapshot):
    numeric_features = [
        snapshot["My_Mana"],
        snapshot["Their_Mana"],
        snapshot["My_Health"],
        snapshot["Their_Health"]
    ]
    seen_vec = encode_cards(snapshot["Their_Card_Types_Seen"])
    visible_vec = encode_cards(snapshot["Their_Currently_Visible_Cards"])
    active_vec = encode_cards(snapshot["Active_Cards"])
    deck_vec = encode_cards(snapshot["Deck"])

    return np.array(numeric_features + seen_vec + visible_vec + active_vec + deck_vec, dtype=float)
import torch


encode_cards(["Ghost"])



[0, 1, 0, 0, 0, 0, 0, 0, 0]

In [2]:
# -----------------------------
# Game controller
# -----------------------------
class Game:
    def __init__(self, player_1, player_2, length=10, prnt=True, offset=[0,2]):
        self.turn = 0
        self.prnt = prnt
        self.moves_played = 0
        self.length = length
        self.game_over = False
        self.players = [player_1, player_2]
        player_1.game = self
        player_2.game = self
        player_1.team = 0
        player_2.team = 1
        player_2.base_health += offset[0]
        player_2.mana += offset[1]

    def change_turn(self):
        self.turn = 1 - self.turn

    def play(self):
        while self.moves_played < self.length and not self.game_over:
            player = self.players[self.turn]
            opponent = self.players[self.turn-1]
            if self.prnt:
                print(f"\n--- Turn {self.moves_played}, Player {self.turn+1} ---")
            player.make_move(self)
            
        return self.end_game()

    def end_game(self):
        p1, p2 = self.players
        if p1.base_health > p2.base_health:
            return 1
        elif p1.base_health < p2.base_health:
            return -1
        return 0
    

In [3]:
# -----------------------------
# GamePlayer
# -----------------------------
class GamePlayer:
    def __init__(self, difficulty, game, deck, decision_maker):
        self.difficulty = difficulty
        self.game = game
        # Start with the full deck as "largedeck"
        self.largedeck = deepcopy(deck)
        self.deck = []  # current hand (max 5)
        self.active_cards = []
        self.decision_maker = decision_maker
        self.team = None
        self.available_squares = [Square(i) for i in range(4)]
        self.mana = 20
        self.base_health = 50
        # draw initial hand
        self.draw_cards()
        snapshot = {
           
            "My_Mana": self.mana,
            "Their_Mana": self.mana,
            "My_Health": self.base_health,
            "Their_Health": self.base_health,
            "Their_Card_Types_Seen": [],
            "Their_Currently_Visible_Cards": [],
            "Active_Cards":[],
            "Deck":[c.name for c in self.deck]
        }
        self.information=[snapshot_to_vector(snapshot)]
        self.known_enemy_types = set()
        self.passive_effectors=[]
        
    def clone(self):
        return deepcopy(self)

    def draw_cards(self):
        """Refill hand up to 5 cards from largedeck."""
        while len(self.deck) < 5 and self.largedeck:
            idx = np.random.choice(len(self.largedeck))
            self.deck.append(self.largedeck.pop(idx))

    def refresh_deck(self):
        """Call this at the start of a turn to ensure 5 cards in hand."""
        self.draw_cards()

    def get_possible_moves(self, game):
        moves = [EndTurnMove(self)]
        # Only consider cards that are alive
        extend_deck = [c for c in self.deck + self.active_cards]

        for card in extend_deck:
            moves.extend(card.get_possible_moves(game,self))

        return moves

    def make_move(self, game):
        moves = self.get_possible_moves(game)
        if moves:
            move = self.decision_maker.choose_move(moves,torch.tensor(self.information, dtype=torch.float32))
            move.execute(game)
    

    def record_information(self, game):
        opponent = game.players[1 - self.team]
        visible_enemy_cards = [c for c in opponent.active_cards if not getattr(c, "hidden", False)]

        # update memory of seen types
        self.known_enemy_types.update(c.name for c in visible_enemy_cards)

        snapshot = {
           
            "My_Mana": self.mana,
            "Their_Mana": opponent.mana,
            "My_Health": self.base_health,
            "Their_Health": opponent.base_health,
            "Their_Card_Types_Seen": list(self.known_enemy_types),
            "Their_Currently_Visible_Cards": list({c.name for c in visible_enemy_cards}),
            "Active_Cards":[c.name for c in self.active_cards],
            "Deck":[c.name for c in self.deck]
        }
      
        self.information.append(snapshot_to_vector(snapshot))
    def invigorate(self):
        for card in self.active_cards:
            card.exhausted=False
    def employ_effects(self):
        for effect in self.passive_effectors:
            effect.tick(self)

In [4]:
class Effect:
    def __init__(self, owner, source=None):
        self.owner = owner      # usually the player
        self.source = source    # the troop or item that caused this effect
        self.active = True

    def tick(self, effects):
        pass


class ManaTowerEffect(Effect):
    def tick(self,player):
        
        print(f"{self.source.name} grants +1 mana.")
        self.owner.mana += 1

In [5]:
from abc import ABC, abstractmethod
from copy import deepcopy

# -----------------------------
# Card base class
# -----------------------------
class Card(ABC):
    def __init__(self, type_, team, mana_cost, name="", hidden=False):
        self.type = type_
        self.team = team
        self.mana_cost = mana_cost
        self.name = name
        self.hidden = hidden
        self.deployed = False
        self.dead = False
        self.square = None
        self.exhausted=False
        self.health=0
        self.initial_health=0
    @abstractmethod
    def get_possible_moves(self, game, player):
        pass
    def die(self, player):
        """Troop dies and returns to largedeck for reuse."""
        
        self.deployed = False
        if self in player.active_cards:
            player.active_cards.remove(self)
        if self in player.deck:
            player.deck.remove(self)
        player.largedeck.append(self)  # Recycle the card
        self.square = None
        self.health=self.initial_health
# -----------------------------
# WizardCard
# -----------------------------
class WizardCard(Card):
    def __init__(self, team):
        super().__init__(type_="troop", team=team, mana_cost=6,name="Wizard")
        self.initial_health=10
        self.health = 10
        self.hidden=False
        self.exhausted=False
        self.ability_cost=6

    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed and player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployTroopMove(self, sq,player))
        else:
            opponent = game.players[1 - player.team]
            if player.mana >= self.ability_cost and not self.exhausted and self.deployed:
                for sq in opponent.available_squares:
                    moves.append(FireballMove(self, sq,player))
        return moves

# -----------------------------
# WizardCard
# -----------------------------
class GhostCard(Card):
    def __init__(self, team):
        super().__init__(type_="troop", team=team, mana_cost=6, name="Ghost")
        self.health = 6
        self.initial_health=6
        self.hidden=True
        self.exhausted=False
        self.ability_cost=2
        self.damage=1
        
    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed and player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployTroopMove(self, sq,player))
        else:
            opponent = game.players[1 - player.team]
            if player.mana >= self.ability_cost and not self.exhausted and self.deployed:
                for sq in opponent.available_squares:
                    moves.append(HauntMove(self,sq,self.damage,self.ability_cost,player))

        return moves
    def die(self,player):
        super().die(player)
        self.ability_cost=2
        self.damage=1
        

class HealCard(Card):
    def __init__(self, team):
        super().__init__(type_="spell", team=team, mana_cost=4, name="HealSpell")
        self.health = 1000
        self.hidden=True
        self.exhausted=False
        self.ability_cost=1000
    def get_possible_moves(self, game, player):
        self.player = player
        

        opponent = game.players[1 - player.team]
        if player.mana >= 4 and not self.exhausted:
             

            return [HealMove(self,player)]
        return []
class ExorcismCard(Card):
    def __init__(self, team):
        super().__init__(type_="spell", team=team, mana_cost=4, name="ExorcismSpell")
        self.health = 1000
        self.hidden=True
        self.exhausted=False
        
    def get_possible_moves(self, game, player):
        self.player = player
        

        opponent = game.players[1 - player.team]
        if player.mana >= self.mana_cost and not self.exhausted:
             

            return [ExorcismMove(self,player)]
        return []

In [6]:
# -----------------------------
# ExecutionerCard
# -----------------------------
class ExecutionerCard(Card):
    def __init__(self, team):
        super().__init__(type_="troop", team=team, mana_cost=6,name="Executioner")
        self.initial_health=12
        self.health = 12
        self.hidden=False
        self.exhausted=False
        self.ability_cost=6
        self.execution_threshold=200

    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed and player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployTroopMove(self, sq,player))
        else:
            opponent = game.players[1 - player.team]
            if player.mana >= self.ability_cost and not self.exhausted and self.deployed:
                for sq in [square for square in opponent.available_squares if not square.empty and not square.troop.hidden]:
                    if sq.troop.health<=.01*self.execution_threshold*sq.troop.initial_health:
                        moves.append(ExecutionerMove(self, sq,player))
        return moves

# -----------------------------
# ExecutionerCard
# -----------------------------

In [7]:
class ManaTowerCard(Card):
    def __init__(self,team):
        super().__init__(type_="troop",team=team,mana_cost=3,name="ManaTower")
        self.initial_health=4
        self.health=self.initial_health
        self.hidden=False
        self.exhausted=False
        self.ability_cost=6
        self.effect=None
        self.deployed=False
    def get_possible_moves(self, game, player):
        self.player = player
        moves = []
        if not self.deployed and player.mana>=self.mana_cost:
            for sq in player.available_squares:
                if sq.empty:
                    moves.append(DeployManaTowerMove(self, sq,player))
        return moves
    def die(self,player):
         
        super().die(player)
        player.passive_effectors.remove(self.effect)
        

In [8]:
# -----------------------------
# Square and BuffSquare
# -----------------------------
class Square:
    def __init__(self, index, troop=None):
        self.index = index
        self.troop = troop
        self.empty = troop is None

class BuffSquare:
    def __init__(self):
        self.empty = True

In [9]:
# -----------------------------
# Move base class
# -----------------------------
class Move(ABC):
    @abstractmethod
    def execute(self, game):
        pass

# -----------------------------
# DeployTroopMove
# -----------------------------
class DeployTroopMove(Move):
    def __init__(self, card, square,player):
        self.card = card
        self.square = square
        self.mana_cost = card.mana_cost
        self.player=player
    def execute(self, game):
        player=self.player
        if self.player.mana < self.mana_cost:
            if game.prnt:
                print(f"❌ Not enough mana to deploy {self.card.name}!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        player.mana -= self.mana_cost
        self.square.troop = self.card
        self.square.empty = False
        self.card.deployed = True
        self.card.square = self.square
        if self.card in player.deck:
            player.deck.remove(self.card)
        player.active_cards.append(self.card)
        if game.prnt:
            print(f"🧙 {self.card.name} deployed on square {self.square.index}. Mana: {player.mana}")
class DeployManaTowerMove(Move):
    def __init__(self, card, square,player):
        self.card = card
        self.square = square
        self.card.effect=ManaTowerEffect(player,card)
        self.player=player
        self.mana_cost=self.card.mana_cost
    def execute(self, game):
        player=self.player
        if self.player.mana < self.mana_cost:
            if game.prnt:
                print(f"❌ Not enough mana to deploy {self.card.name}!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        player.mana -= self.mana_cost
        self.square.troop = self.card
        self.square.empty = False
        self.card.deployed = True
        self.card.square = self.square
        if self.card in player.deck:
            player.deck.remove(self.card)
        player.active_cards.append(self.card)
        if game.prnt:
            print(f"🧙 {self.card.name} deployed on square {self.square.index}. Mana: {player.mana}")
        self.player.passive_effectors.append(self.card.effect)
        

class FireballMove(Move):
    def __init__(self, caster, target_square,player):
        self.caster = caster
        self.target_square = target_square
        self.mana_cost = caster.ability_cost
        self.player=player
    def execute(self, game):
        player=self.player
        if player.mana < self.mana_cost and not self.caster.exhausted:
            if game.prnt:
                print("❌ Not enough mana for fireball!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        player.mana -= self.mana_cost
        opponent = game.players[1 - player.team]
        squares = opponent.available_squares
        idx = squares.index(self.target_square)
        self.caster.exhausted=True

        if game.prnt:
            print(f"Player{player.team+1}'s Wizard on square {self.caster.square.index} casts Fireball at enemy square {idx}! Mana left: {player.mana}")

        splash_indices = [i for i in [idx-1, idx, idx+1] if 0 <= i < len(squares)]
        splash_damage = 6 / len(splash_indices)
        overkill = 0

        for i in splash_indices:
            sq = squares[i]
            if not sq.empty and sq.troop and hasattr(sq.troop, "health") and not sq.troop.dead:
                sq.troop.health -= splash_damage
                if game.prnt:
                    print(f"🔥 {splash_damage:.1f} dmg to {sq.troop.name} on square {i} (remaining {sq.troop.health:.1f})")
                if sq.troop.health <= 0:
                    overkill += -sq.troop.health
                    if game.prnt:
                        print(f"💀 {sq.troop.name} on square {i} destroyed!")
                    sq.troop.die(opponent)
                    sq.troop = None
                    sq.empty = True
            else:
                overkill += splash_damage  # hitting empty square counts as overkill

        if overkill > 0:
            opponent.base_health -= min(overkill,2)
            if game.prnt:
                print(f"🏰 {overkill:.1f} overkill damage to opponent base! Remaining: {opponent.base_health:.1f}")

        if opponent.base_health <= 0:
            if game.prnt:
                print("🎉 Opponent base destroyed! Game over.")
            game.game_over = True
class HauntMove(Move):
    def __init__(self, caster, target_square,damage,mana_cost,player):
        self.caster = caster
        self.target_square = target_square
        self.mana_cost = mana_cost
        self.damage=damage
        self.player=player
    def execute(self, game):
        player=self.player
        if player.mana < self.mana_cost and not self.caster.exhausted:
            if game.prnt:
                
                print("❌ Not enough mana for haunt!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        player.mana -= self.mana_cost
        opponent = game.players[1 - player.team]
        squares = opponent.available_squares
        idx = squares.index(self.target_square)
        self.caster.exhausted=True

        if game.prnt:
            print(f"Player{player.team+1}'s Ghost on square {self.caster.square.index} casts Haunt at enemy square {idx}! Mana left: {player.mana}")

        
        if not self.target_square.empty:
            self.target_square.troop.health-=self.damage
            if game.prnt:
                print(f" {self.damage} damage to opponent at square {idx} Remaining: {self.target_square.troop.health:.1f}")
            if self.target_square.troop.health<=0:
                if game.prnt:
                    print(f" {self.target_square.troop.name} at square {idx} has died!")
                self.target_square.troop.die(opponent)
                self.target_square.troop = None
                self.target_square.empty = True
                
        
            
            
        

        
        if game.prnt:
            print(f" {self.damage} damage to opponent base! Remaining: {opponent.base_health:.1f}")    
        self.caster.damage+=1
        self.caster.ability_cost+=2
        opponent.base_health-=self.damage
        

        if opponent.base_health <= 0:
            if game.prnt:
                print("🎉 Opponent base destroyed! Game over.")
            game.game_over = True
class HealMove(Move):
    def __init__(self,card,player):
        self.card=card
        self.player=player
        self.mana_cost = 4

    def execute(self, game):
        self.player.mana-=self.mana_cost
        if game.prnt:
            print(f"Player {self.player.team+1} casts heal!")
        if self.player.active_cards:
            for card in self.player.active_cards:
                card.health = card.health+2
                if game.prnt and card.square:
                    print(f"Player {self.player.team+1}'s {card.name} on square {card.square.index} is now {card.health} hp!")
        else:
            self.player.base_health+=2
            print(f"Player {self.player.team+1}'s base is healed! Remaining Health:{self.player.base_health} hp!")
            
                  
                
        
            
        self.card.die(self.player)
class ExorcismMove(Move):
    def __init__(self,card,player):
        self.card=card
        self.player=player
        self.mana_cost = card.mana_cost
    def execute(self, game):
        opponent=game.players[1-self.player.team]
        self.player.mana-=self.mana_cost
        if game.prnt:
            print(f"Player {self.player.team+1} casts Exorcism!")
        if opponent.active_cards:
            for card in opponent.active_cards:
                if card.name=="Ghost":
                    if game.prnt and card.square:
                        print(f"Player {opponent.team+1}'s {card.name} on square {card.square.index} has been Exorcised!")
        
                    card.die(opponent)
        else:
            print(f"No ghosts deployed!")
           
            
                  
                
        
            
        self.card.die(self.player)

# -----------------------------
# EndTurnMove
# -----------------------------
class EndTurnMove(Move):
    def __init__(self, player):
        self.player = player
        

    def execute(self, game):
        # End the current player's turn first
        game.change_turn()
        game.moves_played += 1

        # Refresh the new player's deck and info
        new_player = game.players[game.turn]
        old_player = game.players[1 - game.turn]

        old_player.mana += 2
        old_player.refresh_deck()

        # Now the new player records information before acting
        new_player.invigorate()
        new_player.refresh_deck()
        new_player.record_information(game)
        

        if game.prnt:
            print(f"Player {old_player.team+1} ends turn and gains 2 mana.")
        old_player.employ_effects()
        if game.prnt:
            print(f"Player {old_player.team+1} has {old_player.mana} mana remaining.")

In [10]:
class ExecutionerMove(Move):
    def __init__(self, caster, target_square,player):
        self.caster = caster
        self.target_square = target_square
        self.mana_cost = caster.ability_cost
        self.player=player
        self.execution_threshold=caster.execution_threshold
    def execute(self, game):
        executionThreshhold = self.execution_threshold ######<--PERCENT health threshhold for execution to instantly kill enemy unit-->#####
        sq = self.target_square
        player=self.player
        if player.mana < self.mana_cost and not self.caster.exhausted:
            if game.prnt:
                print("❌ Not enough mana/weak targets for execution!")
                print(f" Mana needed: {self.mana_cost} Current Mana: {player.mana}")
            return
        if game.prnt:
            print(f"Player{player.team+1}'s Executioner on square {self.caster.square.index} casts Execute at enemy square {self.target_square.index}! ")
        if sq.empty or not sq.troop or not hasattr(sq.troop, "health") or sq.troop.dead:
            if game.prnt:
                print("❌ No targets for execution!")
            return
        if self.target_square.troop.health >= self.target_square.troop.initial_health * (executionThreshhold * 0.01) :
            if game.prnt:
                print("Target is not weak enough for Executioner to execute")
            return
        if game.prnt:
            print(f"Executioner executes {self.target_square.troop.name}  !!! OWNED!!!!")
        
        player.mana -= self.mana_cost
        opponent = game.players[1 - player.team]
        squares = opponent.available_squares
        idx = squares.index(self.target_square)
        self.caster.exhausted=True
        self.target_square.troop.die(opponent)
        self.target_square.troop = None
        self.target_square.empty = True
        if game.prnt:
            print(f"Mana left: {player.mana}")
       

        if opponent.base_health <= 0:
            if game.prnt:
                print("🎉 Opponent base destroyed! Game over.")
            game.game_over = True

In [11]:
# -----------------------------
# DecisionMaker (random for now)
# -----------------------------
class DecisionMaker:
    def __init__(self, difficulty, json_data=None):
        self.difficulty = difficulty
        self.json_data = json_data

    def choose_move(self, possible_moves,information):
        player=possible_moves[0].player
        if self.difficulty==0:
            return EndTurnMove(player)
        return np.random.choice(possible_moves)

In [12]:
# -----------------------------
# Example simulation
# -----------------------------
def simulate(player1, player2, length=20, prnt=True, offset=[0,0]):
    p1a = player1.clone()
    p2a = player2.clone()
    p1b = player1.clone()
    p2b = player2.clone()
    game1 = Game(p1a, p2a, length, prnt, offset)
    game2 = Game(p2b, p1b, length, prnt, offset)
    return [game1.play(), -game2.play()]

In [13]:
from collections import Counter
def make_deck(team=0, size=20, cards=None, prnt=True, limits={}):

    l = []

    if not cards:
        cards = [WizardCard, GhostCard, HealCard, ExorcismCard, ExecutionerCard,ManaTowerCard]

    # Default: no limits (infinite)
    for card in cards:
        if card not in limits:
    
            limits[card] = float('inf')

    # Keep count of how many of each card we've added
    counts = {c: 0 for c in cards}

    while len(l) < size:
        # Choose from cards that haven't hit their limit yet
        available = [c for c in cards if counts[c] < limits[c]]
        if not available:
            raise ValueError("No more cards available — all limits reached before deck filled.")

        card_cls = np.random.choice(available)
        l.append(card_cls(team=team))
        counts[card_cls] += 1

    if prnt:
        print_deck(l)
    return l


def print_deck(deck=None):
    if not deck:
        deck=make_deck()

    print(Counter([item.name for item in deck]))
deck=make_deck(limits={WizardCard:2,
                      GhostCard:3})

Counter({'HealSpell': 5, 'ExorcismSpell': 4, 'ManaTower': 3, 'Ghost': 3, 'Executioner': 3, 'Wizard': 2})


from IPython.display import display, HTML

display(HTML('''
<style>
.output_scroll {
    height: auto !important;
    max-height: none !important;
}
</style>
'''))

In [14]:
results=[]
deck1 = make_deck(0,limits={
                      GhostCard:1,HealCard:1,ExorcismCard:1})
deck2 = make_deck(1)

Counter({'Executioner': 7, 'ManaTower': 6, 'Wizard': 4, 'Ghost': 1, 'HealSpell': 1, 'ExorcismSpell': 1})
Counter({'Wizard': 5, 'ManaTower': 4, 'Ghost': 4, 'ExorcismSpell': 3, 'Executioner': 2, 'HealSpell': 2})


In [15]:
for a in range(50):



    player1 = GamePlayer(difficulty=1, game=None, deck=deck1, decision_maker=DecisionMaker(1))
    player2 = GamePlayer(difficulty=1, game=None, deck=deck2, decision_maker=DecisionMaker(1))
    simulationresult = simulate(player1, player2,length=40,prnt=True,offset=[0,0])
    results.extend(simulationresult)# Run a test


--- Turn 0, Player 1 ---
Player 1 ends turn and gains 2 mana.
Player 1 has 22 mana remaining.

--- Turn 1, Player 2 ---
🧙 Wizard deployed on square 1. Mana: 14

--- Turn 1, Player 2 ---
🧙 ManaTower deployed on square 3. Mana: 11

--- Turn 1, Player 2 ---
Player2's Wizard on square 1 casts Fireball at enemy square 1! Mana left: 5
🏰 6.0 overkill damage to opponent base! Remaining: 48.0

--- Turn 1, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 8 mana remaining.

--- Turn 2, Player 1 ---
🧙 Ghost deployed on square 1. Mana: 16

--- Turn 2, Player 1 ---
🧙 ManaTower deployed on square 3. Mana: 13

--- Turn 2, Player 1 ---
Player1's Ghost on square 1 casts Haunt at enemy square 2! Mana left: 11
 1 damage to opponent base! Remaining: 50.0

--- Turn 2, Player 1 ---
🧙 Executioner deployed on square 0. Mana: 5

--- Turn 2, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 1 has 8 mana remaining.

--- Turn 3, Player 2 ---

C:\Users\allen\AppData\Local\Temp\ipykernel_992\555316332.py:60: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_new.cpp:257.)
  move = self.decision_maker.choose_move(moves,torch.tensor(self.information, dtype=torch.float32))


ManaTower grants +1 mana.
Player 1 has 4 mana remaining.

--- Turn 15, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 6 mana remaining.

--- Turn 16, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 1 has 8 mana remaining.

--- Turn 17, Player 2 ---
Player2's Executioner on square 2 casts Execute at enemy square 2! 
Executioner executes Wizard  !!! OWNED!!!!
Mana left: 0

--- Turn 17, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 3 mana remaining.

--- Turn 18, Player 1 ---
🧙 Ghost deployed on square 2. Mana: 2

--- Turn 18, Player 1 ---
Player1's Ghost on square 2 casts Haunt at enemy square 3! Mana left: 0
 1 damage to opponent at square 3 Remaining: 11.0
 1 damage to opponent base! Remaining: 47.0

--- Turn 18, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 1 has 4 mana remaining

Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 2 has 6 mana remaining.

--- Turn 6, Player 1 ---
Player1's Ghost on square 2 casts Haunt at enemy square 2! Mana left: 0
 2 damage to opponent at square 2 Remaining: 1.0
 2 damage to opponent base! Remaining: 48.0

--- Turn 6, Player 1 ---
Player 1 ends turn and gains 2 mana.
Player 1 has 2 mana remaining.

--- Turn 7, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 2 has 10 mana remaining.

--- Turn 8, Player 1 ---
Player 1 ends turn and gains 2 mana.
Player 1 has 4 mana remaining.

--- Turn 9, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 2 has 14 mana remaining.

--- Turn 10, Player 1 ---
Player1's Ghost on square 0 casts Haunt at enemy square 0! Mana left: 0
 2 damage to opponent at square 0 Remaining: 10.0
 2 damage to opponent base! Remaining: 46.0

--- Turn 10, 

--- Turn 33, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 4 mana remaining.

--- Turn 34, Player 1 ---
Player 1 casts Exorcism!

--- Turn 34, Player 1 ---
Player 1 ends turn and gains 2 mana.
Player 1 has 2 mana remaining.

--- Turn 35, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 7 mana remaining.

--- Turn 36, Player 1 ---
Player 1 ends turn and gains 2 mana.
Player 1 has 4 mana remaining.

--- Turn 37, Player 2 ---
Player2's Wizard on square 2 casts Fireball at enemy square 2! Mana left: 1
🔥 2.0 dmg to Ghost on square 3 (remaining 4.0)
🏰 4.0 overkill damage to opponent base! Remaining: 44.0

--- Turn 37, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 4 mana remaining.

--- Turn 38, Player 1 ---
Player 1 ends turn and gains 2 mana.
Player 1 has 6 mana remaining.

--- Turn 39, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Playe

...

🧙 Ghost deployed on square 3. Mana: 2

--- Turn 3, Player 2 ---
Player2's Ghost on square 3 casts Haunt at enemy square 1! Mana left: 0
 1 damage to opponent at square 1 Remaining: 9.0
 1 damage to opponent base! Remaining: 50.0

--- Turn 3, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 3 mana remaining.

--- Turn 4, Player 1 ---
🧙 Executioner deployed on square 2. Mana: 0

--- Turn 4, Player 1 ---
Player 1 ends turn and gains 2 mana.
Player 1 has 2 mana remaining.

--- Turn 5, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 6 mana remaining.

--- Turn 6, Player 1 ---
Player 1 ends turn and gains 2 mana.
Player 1 has 4 mana remaining.

--- Turn 7, Player 2 ---
Player2's Ghost on square 3 casts Haunt at enemy square 0! Mana left: 2
 2 damage to opponent at square 0 Remaining: 10.0
 2 damage to opponent base! Remaining: 49.0

--- Turn 7, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +

ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 1 has 5 mana remaining.

--- Turn 11, Player 2 ---
Player2's Ghost on square 0 casts Haunt at enemy square 1! Mana left: 3
 2 damage to opponent at square 1 Remaining: 2.0
 2 damage to opponent base! Remaining: 48.0

--- Turn 11, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 6 mana remaining.

--- Turn 12, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 1 has 9 mana remaining.

--- Turn 13, Player 2 ---
Player 2 casts Exorcism!

--- Turn 13, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 5 mana remaining.

--- Turn 14, Player 1 ---
Player1's Wizard on square 3 casts Fireball at enemy square 1! Mana left: 3
🔥 2.0 dmg to Ghost on square 0 (remaining 4.0)
🔥 2.0 dmg to Ghost on square 1 (remaining 2.0)
🔥 2.0 dmg to Executioner on square 2 (remaining 8.0)

--- Turn 14, Player 1 ---
Player 

Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 2 has 7 mana remaining.

--- Turn 4, Player 1 ---
Player1's Ghost on square 3 casts Haunt at enemy square 3! Mana left: 5
 2 damage to opponent at square 3 Remaining: 8.0
 2 damage to opponent base! Remaining: 51.0

--- Turn 4, Player 1 ---
🧙 ManaTower deployed on square 2. Mana: 2

--- Turn 4, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 1 has 6 mana remaining.

--- Turn 5, Player 2 ---
Player2's Wizard on square 3 casts Fireball at enemy square 3! Mana left: 1
🔥 3.0 dmg to ManaTower on square 2 (remaining 1.0)
🔥 3.0 dmg to Ghost on square 3 (remaining 3.0)

--- Turn 5, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 2 has 5 mana remaining.

--- Turn 6, Player 1 ---
Player 1 casts Exorcism!

--- Turn 6, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grant

Player 1 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 1 has 7 mana remaining.

--- Turn 29, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 7 mana remaining.

--- Turn 30, Player 1 ---
Player1's Wizard on square 3 casts Fireball at enemy square 3! Mana left: 1
🔥 3.0 dmg to Ghost on square 2 (remaining 3.0)
🔥 3.0 dmg to ManaTower on square 3 (remaining 1.0)

--- Turn 30, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 1 has 4 mana remaining.

--- Turn 31, Player 2 ---
Player2's Wizard on square 1 casts Fireball at enemy square 2! Mana left: 1
🔥 2.0 dmg to Ghost on square 1 (remaining 4.0)
🔥 2.0 dmg to ManaTower on square 2 (remaining 3.0)
🔥 2.0 dmg to Wizard on square 3 (remaining 10.0)

--- Turn 31, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 2 has 4 mana remaining.

--- Turn 32, Player 1 ---
🧙 ManaTower deployed on square 0. Mana: 1

--- Turn 32, Player 1 

Player2's Executioner on square 0 casts Execute at enemy square 0! 
Executioner executes ManaTower  !!! OWNED!!!!
Mana left: 41

--- Turn 39, Player 2 ---
Player 2 ends turn and gains 2 mana.
ManaTower grants +1 mana.
ManaTower grants +1 mana.
Player 2 has 45 mana remaining.

--- Turn 0, Player 1 ---
🧙 Wizard deployed on square 2. Mana: 14

--- Turn 0, Player 1 ---
Player1's Wizard on square 2 casts Fireball at enemy square 2! Mana left: 8
🏰 6.0 overkill damage to opponent base! Remaining: 48.0

--- Turn 0, Player 1 ---
🧙 ManaTower deployed on square 0. Mana: 5

--- Turn 0, Player 1 ---
Player 1 casts Exorcism!
No ghosts deployed!

--- Turn 0, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grants +1 mana.
Player 1 has 4 mana remaining.

--- Turn 1, Player 2 ---
Player 2 ends turn and gains 2 mana.
Player 2 has 22 mana remaining.

--- Turn 2, Player 1 ---
🧙 ManaTower deployed on square 1. Mana: 1

--- Turn 2, Player 1 ---
Player 1 ends turn and gains 2 mana.
ManaTower grant

In [16]:

print("1=Win,-1=Loss,0=Draw")
print(Counter(results))

1=Win,-1=Loss,0=Draw
Counter({1: 51, -1: 48, 0: 1})


In [17]:
print("The following is the list of snapshots")
print("Columns contain both players' health, both players' mana, and one-hot encoded lists of cards in hand.")

player2.information

The following is the list of snapshots
Columns contain both players' health, both players' mana, and one-hot encoded lists of cards in hand.


[array([20., 20., 50., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,
         1.])]

In [18]:
np.shape(player2.information)

(1, 40)

In [19]:
torch.tensor(player2.information, dtype=torch.float32)

tensor([[20., 20., 50., 50.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
          0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  1.]])

In [20]:
a=[3,2,6]
[x for x in a if x>2]

[3, 6]